In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import Figure
import numpy as np
import rasterio as rio
from rasterio.plot import show
from rasterio.merge import merge
from pathlib import Path
import richdem as rd

In [ ]:
#data obtained from www.lat-long.com
volc_lat_lon = {}
volc_lat_lon['Mt_Rainier'] = [46.8528857, -121.7603744, 4392.5]
volc_lat_lon['Mt_Adams'] = [46.202621, -121.4906384, 3743.2]
volc_lat_lon['Mt_Baker'] = [48.7773426,  -121.8132008, 3287.6]
# change the lat and lon of mt st helens to the middle of the dome instead of the highest point
volc_lat_lon['Mt_St_Helens'] =[46.200472222222224,-122.18883611111112,2549] #[46.1912, -122.1944, 2549]
volc_lat_lon['Glacier_Peak'] = [48.1112273, -121.1139922, 3213]
volc_lat_lon['Crater_Lake']=[42.907745, -122.143494, 1883]
volc_lat_lon['Mt_Hood']=[45.373221, -121.696509, 3428.7]
volc_lat_lon['Newberry']=[43.7220653, -121.2344654, 2435]

In [ ]:
for name in volc_lat_lon:
    path = Path(name)
    path.mkdir(parents=True, exist_ok=True)
    output_path = 'DEM_data/'+str(name)+'/mosaic_'+str(name)+'.tif'
    volc_files = list(path.iterdir())
    to_mosiac = []
    for d in volc_files:
        raster = rio.open(d)
        to_mosiac.append(raster)
    mosaic, output = merge(to_mosiac)
    output_meta = raster.meta.copy()
    output_meta.update(
        {'driver': 'GTiff',
            'height': mosaic.shape[1],
            'width': mosaic.shape[2],
            'transform': output,})
    with rio.open(output_path, 'w', **output_meta) as m:
        m.write(mosaic)
    break

In [ ]:
name = 'Mt_Baker'
path = Path('DEM_data/'+name)
#path.mkdir(parents=True, exist_ok=True)
output_path = 'DEM_data/'+str(name)+'/mosaic_'+str(name)+'.tif'
volc_files = list(path.iterdir())
to_mosiac = []
for d in volc_files:
    raster = rio.open(d)
    to_mosiac.append(raster)
mosaic, output = merge(to_mosiac)
output_meta = raster.meta.copy()
output_meta.update(
    {'driver': 'GTiff',
        'height': mosaic.shape[1],
        'width': mosaic.shape[2],
        'transform': output,})
with rio.open(output_path, 'w', **output_meta) as m:
    m.write(mosaic)

In [ ]:
dem = rio.open('Mt_Baker/mosaic_Mt_Baker.tif')
dem_array = dem.read(1).astype('float64')

volc = rd.rdarray(dem_array, no_data=-9999)
slope = rd.TerrainAttribute(volc,attrib = 'slope_riserun')
aspect = rd.TerrainAttribute(volc, attrib = 'aspect')

color_scheme = 'jet'
fig1 = rd.rdShow(volc, axes=True, cmap='jet', figsize=(15,8))
fig2 = rd.rdShow(slope, axes=False, cmap=color_scheme, figsize=(15,8))
fig3 = rd.rdShow(aspect, axes=False, cmap=color_scheme, figsize=(15,8))


In [ ]:
#practice plotting something else over the top of this plot
fig3 = plt.figure(figsize = (15,8), dpi=80)
rd.rdShow(aspect, axes=False, cmap=color_scheme, figsize=(15,8))
x = np.array(range(300))
plt.plot(x, x, ls='dotted', linewidth=2, color='k')
plt.show()
